In [ ]:
# Cell 1
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Cell 2
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Cell 3
import pandas as pd

# Load dataset
dataset = pd.read_csv("insurance.csv")

# Display first few rows
print(dataset.head())

# Check for missing values
print("\nMissing Values:\n", dataset.isnull().sum())

# Display dataset info
print("\nDataset Info:\n")
print(dataset.info())

# Display statistical summary of numerical columns
print("\nStatistical Summary:\n")
print(dataset.describe())


In [ ]:
# Cell 3a
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert categorical variables
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})

# One-hot encoding for 'region'
dataset = pd.get_dummies(dataset, columns=['region'], drop_first=True)

# Define features (X) and labels (y)
X = dataset.drop(columns=['expenses'])
y = dataset['expenses']

# Split data into training (80%) and testing (20%) sets
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
train_dataset[['age', 'bmi', 'children']] = scaler.fit_transform(train_dataset[['age', 'bmi', 'children']])
test_dataset[['age', 'bmi', 'children']] = scaler.transform(test_dataset[['age', 'bmi', 'children']])

# Display processed data
print(train_dataset.head())
print("\nTrain dataset shape:", train_dataset.shape)
print("Test dataset shape:", test_dataset.shape)


In [ ]:
# Cell 3b
# Build the neural network model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer (regression problem, so 1 neuron)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',  # Mean Squared Error
              metrics=['mae', 'mse'])  # Mean Absolute Error & Mean Squared Error

# Train the model
history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=1)

# Display training summary
print("\nTraining complete!")


In [ ]:
# Cell 4
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
